In [23]:
import pandas as pd

In [24]:
df=pd.read_csv('../data/raw/1000dataset.csv')

In [25]:
df.head()

,track_id,artists,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,360Wr96ywrCQq4kTAJ8Pq6,['Melanie Martinez'],80,220013,0.576,0.593,9,-6.024,0,0.0306,0.04670,0.000767,0.237,0.3730,157.993,4,electro
1,3yqoD9SYDfwvNEdU2NEpFx,['Rovalio'],50,146891,0.586,0.402,0,-10.913,0,0.0301,0.42100,0.903000,0.135,0.1550,150.012,4,electro
2,2ygvZOXrIeVL4xZmAWJT2C,['Billie Eilish'],72,208155,0.444,0.309,8,-10.956,1,0.0620,0.79500,0.132000,0.352,0.0875,104.745,4,electro
3,6HMT7AzGA5wmn5m2t1BuZR,"['Alan Walker', 'Steve Aoki', 'ISÁK', 'Omar No...",54,216053,0.521,0.819,6,-7.267,0,0.0413,0.00911,0.002270,0.086,0.1840,89.952,4,electro
4,0uLPKDuIhkIhttiwezNdiG,['Alec Benjamin'],66,171539,0.787,0.558,0,-5.237,1,0.0414,0.66400,0.000000,0.347,0.7060,94.049,4,electro


In [26]:
# keep only track_id and track_genre
dfg = df[['track_id', 'track_genre']]

In [27]:
dfg.info()
dfg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   track_id     1000 non-null   object
 1   track_genre  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


,track_id,track_genre
0,360Wr96ywrCQq4kTAJ8Pq6,electro
1,3yqoD9SYDfwvNEdU2NEpFx,electro
2,2ygvZOXrIeVL4xZmAWJT2C,electro
3,6HMT7AzGA5wmn5m2t1BuZR,electro
4,0uLPKDuIhkIhttiwezNdiG,electro


In [50]:
## Perform one-hot encoding
#dfg_e = pd.get_dummies(dfg, columns=['track_genre'])
#
## Identify the one-hot-encoded columns
#one_hot_columns = [col for col in dfg_e.columns if 'track_genre_' in col]
#
## Convert only the one-hot-encoded columns to integers
#dfg_e[one_hot_columns] = dfg_e[one_hot_columns].astype(int)
#
## Verify the result
#dfg_e.head()


In [51]:
import os
import pandas as pd

# Path to the folder containing images
image_folder = "../data/raw/1000_mel_spec_seg"

# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

# Create a list to store rows for the new CSV file
image_data = []

# Loop through image files
for image_file in image_files:
    # Extract the track_id from the filename (everything before "_segment")
    track_id = "_".join(image_file.split("_")[:-2])  # Adjust splitting based on naming
    
    # Look up the corresponding genre or class in the dataframe
    if track_id in dfg['track_id'].values:  # Assuming 'track_id' column exists in dfg
        row = dfg[dfg['track_id'] == track_id].iloc[0]
        
        # Extract the class or genre information
        class_info = row['track_genre']  # Replace with one-hot columns if needed
        
        # Add a row to the new data: [image path, class/genre]
        image_data.append([os.path.join(image_folder, image_file), class_info])
    else:
        print(f"Track ID {track_id} not found in the dataframe!")  # Debugging info

# Create a new dataframe for the association file
association_df = pd.DataFrame(image_data, columns=["image_path", "class"])
association_df.head()

,image_path,class
0,../data/raw/1000_mel_spec_seg\000RDCYioLteXcut...,hip-hop
1,../data/raw/1000_mel_spec_seg\000RDCYioLteXcut...,hip-hop
2,../data/raw/1000_mel_spec_seg\000RDCYioLteXcut...,hip-hop
3,../data/raw/1000_mel_spec_seg\000RDCYioLteXcut...,hip-hop
4,../data/raw/1000_mel_spec_seg\000RDCYioLteXcut...,hip-hop


In [52]:
# label encode association_df
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
association_df['class'] = le.fit_transform(association_df['class'])

association_df.to_csv('../data/raw/Limage_class_association.csv', index=False)

In [ ]:
# Perform one-hot encoding
dfg_e = pd.get_dummies(association_df, columns=['class'])

# Identify the one-hot-encoded columns
one_hot_columns = [col for col in dfg_e.columns if 'class' in col]

# Convert only the one-hot-encoded columns to integers
dfg_e[one_hot_columns] = dfg_e[one_hot_columns].astype(int)

# Verify the result
dfg_e.head()


In [45]:

# Save the new dataframe to a CSV file
association_csv_path = "../data/raw/Nimage_class_association.csv"
dfg_e.to_csv(association_csv_path, index=False)

print(f"Association CSV file created: {association_csv_path}")

Association CSV file created: ../data/raw/Nimage_class_association.csv


## Create dataset + dataloader


In [1]:
from collections import defaultdict
from sklearn.model_selection import train_test_split
import pandas as pd
import random
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

# Step 1: Group images by track ID and class
def group_tracks_by_class_and_id(association_csv):
    df = pd.read_csv(association_csv)
    df['track_id'] = df['image_path'].apply(lambda x: "_".join(os.path.basename(x).split("_")[:-2]))
    class_groups = defaultdict(list)
    
    # Group track IDs by their class
    for track_id, group in df.groupby('track_id'):
        track_class = group.iloc[0]['class']
        class_groups[track_class].append(track_id)
    
    return class_groups, df

# Step 2: Split track IDs for each class
def split_tracks_by_class(class_groups, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    train_ids, val_ids, test_ids = [], [], []
    
    for track_class, track_ids in class_groups.items():
        random.shuffle(track_ids)  # Shuffle track IDs within the class
        
        # Perform splits
        train, temp = train_test_split(track_ids, test_size=(1 - train_ratio))
        val, test = train_test_split(temp, test_size=(test_ratio / (test_ratio + val_ratio)))
        
        # Append to respective splits
        train_ids.extend(train)
        val_ids.extend(val)
        test_ids.extend(test)
    
    return train_ids, val_ids, test_ids


In [2]:
# Step 3: Create a custom PyTorch Dataset
class SpectrogramDataset(Dataset):
    def __init__(self, df, track_ids, transform=None):
        self.data = df[df['track_id'].isin(track_ids)]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = row['image_path']
        label = row['class']
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [3]:
# Step 4: Create DataLoaders
def create_balanced_dataloaders(image_folder, association_csv, batch_size=32):
    # Group by class and track ID
    class_groups, df = group_tracks_by_class_and_id(association_csv)
    
    # Perform class-balanced splits
    train_ids, val_ids, test_ids = split_tracks_by_class(class_groups)
    
    # Define image transformations (e.g., resizing, normalization)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to a consistent size
        transforms.ToTensor(),         # Convert to tensor
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
    ])
    
    # Create datasets
    train_dataset = SpectrogramDataset(df, train_ids, transform=transform)
    val_dataset = SpectrogramDataset(df, val_ids, transform=transform)
    test_dataset = SpectrogramDataset(df, test_ids, transform=transform)
    
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

# Example Usage
image_folder = "../data/raw/1000mel_spec_seg"
association_csv = "../data/raw/Limage_class_association.csv"

train_loader, val_loader, test_loader = create_balanced_dataloaders(image_folder, association_csv)

# Verify the splits
print(f"Number of training samples: {len(train_loader.dataset)}")
print(f"Number of validation samples: {len(val_loader.dataset)}")
print(f"Number of testing samples: {len(test_loader.dataset)}")


Number of training samples: 6424
Number of validation samples: 947
Number of testing samples: 1875


In [4]:
dataset = train_loader.dataset
print(dataset[0])

(tensor([[[-1.0000, -1.0000, -1.0000,  ..., -0.9373, -0.9216, -0.8353],
         [-1.0000, -1.0000, -1.0000,  ..., -0.9373, -0.9216, -0.8353],
         [-1.0000, -1.0000, -1.0000,  ..., -0.9216, -0.9216, -0.8431],
         ...,
         [-0.5686, -0.8745, -0.6863,  ...,  0.9294,  0.9529,  0.9608],
         [-0.7098, -0.9686, -0.9373,  ...,  0.9137,  0.9373,  0.9451],
         [-0.7490, -0.9922, -1.0000,  ...,  0.9137,  0.9294,  0.9451]],

        [[-1.0000, -1.0000, -1.0000,  ..., -0.9686, -0.9765, -0.9608],
         [-1.0000, -1.0000, -1.0000,  ..., -0.9686, -0.9765, -0.9608],
         [-1.0000, -1.0000, -1.0000,  ..., -0.9686, -0.9765, -0.9529],
         ...,
         [-0.8902, -0.9451, -0.9294,  ...,  0.6784,  0.5843,  0.5294],
         [-0.9373, -0.9843, -0.9843,  ...,  0.7255,  0.6549,  0.5843],
         [-0.9529, -0.9922, -1.0000,  ...,  0.7412,  0.6706,  0.6000]],

        [[-0.9686, -0.9686, -0.9686,  ..., -0.8196, -0.8196, -0.6549],
         [-0.9686, -0.9686, -0.9686,  ..., -

# check track_id excistence in the dataloaders

# Model Creation

In [16]:
import torch
from torch import nn
#from audioset_tagging_cnn.models import Cnn14  
import torch.nn as nn
from torchvision import models

# Load a pre-trained model
model = models.resnet18(pretrained=True)

# Modify the final layer for the number of genres 
num_genres = 5  # Replace with the actual number of genres
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),  # Intermediate layer
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 5)  # Output layer for genres
)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Training/Validation loop

In [14]:
import wandb

# Initialize a new W&B run
wandb.init(project='Audio_Class',  
    
    config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 32,
    # Add other hyperparameters as needed
})

# Access the configuration
config = wandb.config


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_accuracy,▁▄▄▅▆▆▇▇██
train_loss,█▆▅▄▃▂▂▁▁▁
val_accuracy,▁▄▂▅██▇▁▄▃
val_loss,▆▄▄▄▁▁▁▅▃█
epoch,10
train_accuracy,0.61473
train_loss,0.89864
val_accuracy,0.41394
val_loss,1.90581


In [24]:
from tqdm import tqdm
import os
import torch
import wandb

# Initialize a new W&B run
wandb.init(project='Audio_Class',  
    
    config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 32,
    # Add other hyperparameters as needed
})

# Access the configuration
config = wandb.config


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_interval=1):
    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct = 0, 0

        # Initialize tqdm progress bar for training
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=True)
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate metrics
            train_loss += loss.item() * images.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()

            # Update tqdm bar
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        train_loss /= len(train_loader.dataset)
        train_accuracy = train_correct / len(train_loader.dataset)
        progress_bar.close()
        
        # Log training metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
        })

        # Validation phase
        model.eval()
        val_loss, val_correct = 0, 0

        # Initialize tqdm progress bar for validation
        progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation", leave=True)
        
        with torch.no_grad():
            for images, labels in progress_bar:
                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Calculate metrics
                val_loss += loss.item() * images.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

                # Update tqdm bar
                progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})
        
        val_loss /= len(val_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)
        progress_bar.close()
        
                # Log validation metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy,
        })

        # Print epoch summary
        print(f"Epoch [{epoch+1}/{num_epochs}]: "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

        # Save the model at specific intervals
        if (epoch + 1) % save_interval == 0:
            save_path = os.path.join("saved_models", f"model_epoch_{epoch+1}.pth")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
            }, save_path)
            print(f"Model saved to {save_path}")


epoch,▁▁▃▃▅▅▆▆██
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▁▄██▆
val_loss,▅▁▃▃█
epoch,5
train_accuracy,0.842
train_loss,0.42554
val_accuracy,0.65576
val_loss,1.14189


In [10]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def test_model(model, test_loader, device, class_names=None):
    """
    Test the trained model on the test dataset.

    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader for the test set.
        device: Torch device (CPU or CUDA).
        class_names: List of class names corresponding to the labels (optional).
    
    Returns:
        None. Prints accuracy, precision, recall, and F1-score.
    """
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Labels are already numeric (encoded 0-4), no need to transform
            # Forward pass
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)  # Get predicted class indices

            # Collect predictions and true labels
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Print results
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")

    # Optional: Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    if class_names:
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.show()


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
save_folder = "../models/classifier"

In [27]:
# Assuming model, train_loader, val_loader, criterion, optimizer are defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")


Training on device: cuda


In [28]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10 - Validation: 100%|██████████| 30/30 [00:13<00:00,  2.17it/s, Loss=0.7075]


Epoch [1/10]: Train Loss: 0.1870, Train Accuracy: 0.9368, Val Loss: 1.0891, Val Accuracy: 0.7254
Model saved to saved_models\model_epoch_1.pth


Epoch 2/10 - Validation: 100%|██████████| 30/30 [00:13<00:00,  2.18it/s, Loss=0.7147]


Epoch [2/10]: Train Loss: 0.1076, Train Accuracy: 0.9626, Val Loss: 1.3618, Val Accuracy: 0.6864
Model saved to saved_models\model_epoch_2.pth


Epoch 3/10 - Validation: 100%|██████████| 30/30 [00:13<00:00,  2.18it/s, Loss=0.6910]


...

Epoch [8/10]: Train Loss: 0.0345, Train Accuracy: 0.9894, Val Loss: 1.5075, Val Accuracy: 0.7191
Model saved to saved_models\model_epoch_8.pth


Epoch 9/10 - Validation: 100%|██████████| 30/30 [00:14<00:00,  2.00it/s, Loss=1.8110]


Epoch [9/10]: Train Loss: 0.0296, Train Accuracy: 0.9896, Val Loss: 1.7748, Val Accuracy: 0.6917
Model saved to saved_models\model_epoch_9.pth


Epoch 10/10 - Validation: 100%|██████████| 30/30 [00:14<00:00,  2.09it/s, Loss=1.0285]

Epoch [10/10]: Train Loss: 0.0581, Train Accuracy: 0.9818, Val Loss: 1.5320, Val Accuracy: 0.7043
Model saved to saved_models\model_epoch_10.pth


In [ ]:
# test model
test_model(model, test_loader, device)

Test Accuracy: 0.6287
Test Precision: 0.6576
Test Recall: 0.6287
Test F1 Score: 0.6376
